In [0]:
%sh
pip install --upgrade pip
pip install wget

Requirement already satisfied: pip in /databricks/python3/lib/python3.7/site-packages (21.3.1)
Requirement already satisfied: wget in /databricks/python3/lib/python3.7/site-packages (3.2)

In [0]:
import datetime
import wget
import requests
import json
import pandas as pd
import numpy as np

Importing the Players and Tournament Datasets

In [0]:

instance_id = "https://api.sportsdata.io/golf/v2/json/Players?key="
key = "c76c6101adbf4b0abb54a7a6eb5ddbb4"
url = f"{instance_id}{key}"

response = requests.get(
url = url)

In [0]:
Players = pd.DataFrame((json.loads(response.text)))

In [0]:
#Players.to_csv('/dbfs/FileStore/karbide/Players.txt')

In [0]:
instance_id = "https://api.sportsdata.io/golf/v2/json/Tournaments?key="
key = "c76c6101adbf4b0abb54a7a6eb5ddbb4"
url = f"{instance_id}{key}"

response = requests.get(
url = url)

Tournaments = pd.DataFrame((json.loads(response.text)))

In [0]:
Tournaments["New_Date"] = pd.to_datetime(Tournaments["StartDate"])

In [0]:
Last_Season = Tournaments.loc[Tournaments["New_Date"]>"2020-09-08"]
Last_Season = Last_Season.loc[Last_Season["New_Date"]<"2021-09-10"]
Last_Season = Last_Season.loc[Last_Season["Name"] != 'QBE Shootout']

In [0]:
#Last_Season.to_csv('/dbfs/FileStore/karbide/Last_Season.txt')

Lets pull just the top 150 players

In [0]:
top150 = spark.read.csv("/FileStore/karbide/top150players.csv")
top150 = top150.toPandas()
top150.columns = ["Id","Name","Rating"]

In [0]:
Players['Full Name'] = Players[['FirstName','LastName']].agg(' '.join, axis=1)

In [0]:
top150Players = Players.merge(top150, how = "inner", left_on= "Full Name", right_on = "Name")
top150Players.shape

Out[116]: (135, 28)

In [0]:
#we lost some players, lets try to see why
top150_list = top150["Name"].tolist()
mergedPlayers_list = top150Players["Name"].tolist()
dropped = [x for x in top150_list if x not in mergedPlayers_list]
droppednames = pd.DataFrame(dropped)
droppednames.columns = ["Name"]
droppednames[["Name1","Name2","Name3"]] = droppednames["Name"].str.split(" ",2,expand=True)


In [0]:
#We lose less players if we use the draft kings names
top150Players2 = Players.merge(top150, how = "inner", left_on= "DraftKingsName", right_on = "Name")
top150Players2.shape

Out[118]: (144, 28)

In [0]:
mergedPlayers_list = top150Players2["Name"].tolist()
dropped = [x for x in top150_list if x not in mergedPlayers_list]
droppednames = pd.DataFrame(dropped)
droppednames.columns = ["Name"]
droppednames[["Name1","Name2","Name3"]] = droppednames["Name"].str.split(" ",2,expand=True)
droppednames

,Name,Name1,Name2,Name3
0,Erik van Rooyen,Erik,van,Rooyen
1,K.H. Lee,K.H.,Lee,None
2,Robert MacIntyre,Robert,MacIntyre,None
3,Sebasti�n Mu�oz,Sebasti�n,Mu�oz,None
4,Cam Davis,Cam,Davis,None
5,Daniel van Tonder,Daniel,van,Tonder


In [0]:
#this is not small enough that we can search 1 by 1
print(Players.loc[Players["FirstName"] == 'Erik'])
Players.at[4443,'DraftKingsName'] = "Erik van Rooyen"

BirthCity BirthDate ... YahooPlayerID Full Name
308 Kalamazoo 1987-10-22T00:00:00 ... 11244.0 Erik Barnes
836 Miami 1979-11-11T00:00:00 ... 3273.0 Erik Compton
1301 None None ... NaN Erik Flores
3245 None None ... NaN Erik Oja
4450 Bellville 1990-02-21T00:00:00 ... 16790.0 Erik Van Rooyen

[5 rows x 25 columns]

In [0]:
print(Players["DraftKingsName"].loc[Players["LastName"] == 'Lee'])
print(top150.loc[top150["Name"]== 'K.H. Lee'])
top150.at[58,"Name"] = "Kyoung-Hoon Lee"

2467 Alex Lee
2468 None
2469 Brandon Lee
2470 Chang Gi Lee
2471 None
2472 Chieh-po Lee
2473 CJ Lee
2474 Craig Lee
2475 Dong-Hwan Lee
2476 None
2477 Danny Lee
2478 None
2479 None
2480 None
2481 Donny Lee
2482 Gayoung Lee
2483 Gyu Min Lee
2484 None
2485 None
2486 None
2487 Hyungjoon Lee
2488 Ilhee Lee
2489 Jaekyeong Lee
2490 None
2491 None
2492 Jihyun Lee
2493 None
2494 Jung Min Lee
2495 Junghwan Lee
2496 Jun-Seok Lee
2497 None
2498 None
2499 None
2500 Kyoung-Hoon Lee
2501 Lucas Lee
2502 None
2503 Min Lee
2504 Min Woo Lee
2505 None
2506 Richard H. Lee
2507 Richard T. Lee
2508 Rodrigo Lee
2509 None
2510 Sam Lee
2511 Sang-hee Lee
2512 None
2513 None
2514 None
2515 Somi Lee
2516 Soomin Lee
2517 None
2518 Taehee Lee
2519 Tain Lee
2520 Walker Lee
2521 Wonjoon Lee
Name: DraftKingsName, dtype: object
 Id Name Rating
58 59 K.H. Lee 2.15

In [0]:
print(Players["DraftKingsName"].loc[Players["FirstName"] == 'Robert'])
Players.at[2688,"DraftKingsName"] = "Robert MacIntyre"

146 Robert Allenby
339 None
625 Robert Calvesbert
668 None
826 Robert Coles
944 None
972 None
1046 Robert Dinwiddie
1153 None
1271 Robert Figueroa
1306 Robert Foley
1403 Robert Gamez
1429 Robert Garrigus
1442 None
1844 Robert Hogan
2107 Robert Kalinowski
2132 Robert Karlsson
2694 Robert Macintyre
2847 None
3031 Robert Moran
3714 Robert Rock
3732 None
4180 Robert Streb
4318 None
4527 None
4649 None
Name: DraftKingsName, dtype: object

In [0]:
print(Players["DraftKingsName"].loc[Players["LastName"] == 'Munoz'])
print(top150.loc[top150["Name"]== 'Sebasti�n Mu�oz'])
top150.at[66,"Name"] = "Sebastian Munoz"

3079 None
3080 Sebastian Munoz
Name: DraftKingsName, dtype: object
 Id Name Rating
66 67 Sebasti�n Mu�oz 1.96

In [0]:
print(Players["DraftKingsName"].loc[Players["LastName"] == 'Davis'])
print(top150.loc[top150["Name"]== 'Cam Davis'])
top150.at[69,"Name"] = "Cameron Davis"

957 Brian Davis
958 Cameron Davis
959 None
960 None
961 None
Name: DraftKingsName, dtype: object
 Id Name Rating
69 70 Cam Davis 1.89

In [0]:
Players.loc[Players["LastName"] == 'Van Tonder']
Players.at[4446,"DraftKingsName"] = "Daniel van Tonder"

In [0]:
top150Players2 = Players.merge(top150, how = "inner", left_on= "DraftKingsName", right_on = "Name")
top150Players2.shape
#now we have all the players

Out[126]: (150, 28)

In [0]:
#the sport data api requires us to input the player and tournament ID for the hole by hole scores, lets see if we can loop and dowload automatically
top150PlayerIDs = top150Players2["PlayerID"].tolist()
Tourney_IDs = Last_Season["TournamentID"].tolist()

In [0]:
#top150Players2.to_csv('/dbfs/FileStore/karbide/top150playersexpanded.txt')

In [0]:
#tournament ID
a = 453
#player ID
b = 40000047

instance_id = "https://api.sportsdata.io/golf/v2/json/PlayerTournamentStatsByPlayer/"
key = "?key=c76c6101adbf4b0abb54a7a6eb5ddbb4"
url = f"{instance_id}{a}/{b}{key}"

response = requests.get(
url = url)

test = json.loads(response.text)

In [0]:
test2 = test['Rounds']
RoundData = pd.DataFrame(test2)

In [0]:
R1Scores = test2[0]["Holes"]
R2Scores = test2[1]["Holes"]
R3Scores = test2[2]["Holes"]
R4Scores = test2[3]["Holes"]
R1Scoresdf = pd.DataFrame(R1Scores)

In [0]:
def holeScore(data):
  ScoresSet = data.drop(["PlayerRoundID","Par","Score","HoleInOne","ToPar"], axis = 1)
  ScoresSet.set_index("Number", inplace = True)
  HoleScores = ScoresSet[ScoresSet == 1].stack()
  
  
  HoleScoresdf = pd.DataFrame(HoleScores).reset_index()
  HoleScoresdf.columns = ["Number","Hole_Score","1"]
  HoleScoresdf.drop(["1"],axis = 1, inplace = True)
  
  Scoresdf = data.merge(HoleScoresdf, on = "Number")
  
  return(Scoresdf)

In [0]:
def numScore(data):
  data["Hole_ScoreNum"] = [-3 if x == "DoubleEagle" else -2 if x == "Eagle" else -1 if x == "Birdie" else 0 if x == "IsPar" else 1 if x == "Bogey" else 2 if x == "DoubleBogey" else 3 for x in data["Hole_Score"]]

In [0]:
R1Scoresdf = holeScore(R1Scoresdf)
numScore(R1Scoresdf)

In [0]:
roundScore = sum(R1Scoresdf['Hole_ScoreNum'])
roundShots = sum(R1Scoresdf['Par'])-roundScore
birdies = sum(R1Scoresdf["Birdie"])

In [0]:
def roundSummary(data,roundNum, playerid, tournamentid):
  roundScore = sum(data['Hole_ScoreNum'])
  roundShots = sum(data['Par'])+roundScore
  doubleeagles = sum(data["DoubleEagle"])
  eagles = sum(data["Eagle"])
  birdies = sum(data["Birdie"])
  pars = sum(data["IsPar"])
  bogeys = sum(data["Bogey"])
  doublebogeys = sum(data['DoubleBogey'])
  worsethandoublebogeys = sum(data['WorseThanDoubleBogey'])
  roundID = data["PlayerRoundID"][0]
  
  roundStats = pd.DataFrame(np.array([[roundScore,roundShots,doubleeagles,eagles,birdies,pars,bogeys,doublebogeys,worsethandoublebogeys,roundID,roundNum,playerid,tournamentid]]),  columns=['RoundScore','RoundShots','DoubleEagles','Eagles','Birdies','Pars','Bogeys','DoubleBogeys','WorseThanDoubleBogeys','PlayerRoundID','RoundNum','PlayerID','TournamentID'])
  return(roundStats)

In [0]:
roundSummary(R1Scoresdf,1,b,a)

,RoundScore,RoundShots,DoubleEagles,Eagles,Birdies,Pars,Bogeys,DoubleBogeys,WorseThanDoubleBogeys,PlayerRoundID,RoundNum,PlayerID,TournamentID
0,-1,69,0,0,4,11,3,0,0,131143,1,40000047,453


In [0]:
def dictToDf(scoresDict,playerid,tournamentid):
  roundDict = scoresDict['Rounds']
  rounddf = pd.DataFrame(roundDict)
  rounddf = rounddf.loc[rounddf["Par"] > 0]
  rounds = rounddf["Number"].tolist()
  dfTournamentRounds =  pd.DataFrame(columns = ['RoundScore','RoundShots','DoubleEagles','Eagles','Birdies','Pars','Bogeys','DoubleBogeys','WorseThanDoubleBogeys','PlayerRoundID','RoundNum','PlayerID','TournamentID'])
  dfTournamentHoles = pd.DataFrame(columns = ['PlayerRoundID', 'Number', 'Par', 'Score', 'ToPar', 'HoleInOne','DoubleEagle', 'Eagle', 'Birdie', 'IsPar', 'Bogey', 'DoubleBogey','WorseThanDoubleBogey', 'Round','Hole_Score', 'Hole_ScoreNum', "Player_ID", "Tournament_ID"])

  for x in rounds:
    roundHoles = roundDict[x-1]["Holes"]
    roundHoles = pd.DataFrame(roundHoles)
    roundHoles = holeScore(roundHoles)
    numScore(roundHoles)
    roundHoles["Round"] = x
    roundHoles["Player_ID"] = playerid
    roundHoles["Tournament_ID"] = tournamentid
    roundstat = roundSummary(roundHoles,x,playerid,tournamentid)
    dfTournamentRounds = pd.concat([dfTournamentRounds,roundstat])
    dfTournamentHoles = pd.concat([dfTournamentHoles,roundHoles])
  
  return(dfTournamentRounds,dfTournamentHoles)
    
    

In [0]:
# testRounds,testHoles = dictToDf(test,b,a)
#print(testHoles.head(20))
#print(testRounds)

Testing a condition if the player didnt play in one the tournament/ the dict is empty

In [0]:
# Example of a empty tournament
url = f"{instance_id}450/40000047{key}"
response = requests.get(url = url)
bool(response.text)

Out[141]: False

In [0]:
# Example of a valid tournament
url = f"{instance_id}451/40000047{key}"
response = requests.get(url = url)
bool(response.text)

Out[142]: True

In [0]:
def allTournaments(playerID,tournamentList,key):
  
  allTournamentRounds = pd.DataFrame(columns = ['RoundScore','RoundShots','DoubleEagles','Eagles','Birdies','Pars','Bogeys','DoubleBogeys','WorseThanDoubleBogeys','PlayerRoundID','RoundNum','PlayerID','TournamentID'])
  allTournamentHoles = pd.DataFrame(columns = ['PlayerRoundID', 'Number', 'Par', 'Score', 'ToPar', 'HoleInOne','DoubleEagle', 'Eagle', 'Birdie', 'IsPar', 'Bogey', 'DoubleBogey','WorseThanDoubleBogey', 'Round','Hole_Score', 'Hole_ScoreNum'])
  
  key = f"?key={key}"
  instance_id = "https://api.sportsdata.io/golf/v2/json/PlayerTournamentStatsByPlayer/"
  
  
  
  for x in tournamentList:
    url = f"{instance_id}{x}/{playerID}{key}"
    response = requests.get(url = url)
    if bool(response.text):
      importdata = json.loads(response.text)
      testDict = importdata['Rounds']
      if bool(testDict):
        xroundDf,xholesDf = dictToDf(importdata,playerID,x)

        allTournamentRounds = pd.concat([allTournamentRounds,xroundDf])
        allTournamentHoles = pd.concat([allTournamentHoles,xholesDf])
  
  return(allTournamentRounds,allTournamentHoles)

In [0]:
#testRounds,testHoles = allTournaments(b,Tourney_IDs,"c76c6101adbf4b0abb54a7a6eb5ddbb4")

In [0]:
def allPlayers(playerList,tournamentList,key):
  allPlayerRounds = pd.DataFrame(columns = ['RoundScore','RoundShots','DoubleEagles','Eagles','Birdies','Pars','Bogeys','DoubleBogeys','WorseThanDoubleBogeys','PlayerRoundID','RoundNum','PlayerID','TournamentID'])
  allPlayerHoles = pd.DataFrame(columns = ['PlayerRoundID', 'Number', 'Par', 'Score', 'ToPar', 'HoleInOne','DoubleEagle', 'Eagle', 'Birdie', 'IsPar', 'Bogey', 'DoubleBogey','WorseThanDoubleBogey', 'Round','Hole_Score', 'Hole_ScoreNum'])
  
  for i in playerList:
    yrounds,yholes = allTournaments(i,tournamentList,key)
    allPlayerRounds = pd.concat([allPlayerRounds,yrounds])
    allPlayerHoles = pd.concat([allPlayerHoles,yholes])
    
  return(allPlayerRounds,allPlayerHoles)


In [0]:
#RoundsDf, HolesDf = allPlayers(top150PlayerIDs,Tourney_IDs,"c76c6101adbf4b0abb54a7a6eb5ddbb4")

/local_disk0/tmp/1637500790570-0/PythonShell.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

 from six.moves import range
/local_disk0/tmp/1637500790570-0/PythonShell.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

 from py4j.java_collections import ListConverter
/local_disk0/tmp/1637500790570-0/PythonShell.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

 import resource

In [0]:
#RoundsDf.to_csv('/dbfs/FileStore/karbide/Rounds.txt')
#HolesDf.to_csv('/dbfs/FileStore/karbide/Holes.txt')

In [0]:
#so I dont have to run the API pull again
RoundsDf = pd.read_csv('/dbfs/FileStore/karbide/Rounds.txt')
HolesDf = pd.read_csv("/dbfs/FileStore/karbide/Holes.txt")

In [0]:
# if a tournament doesnt have at least 100 rounds
def dropSmallTournaments(data,threshold):
  tournamentCounts = data.groupby("TournamentID").size().reset_index(name="counts")
  bigTournaments = tournamentCounts.loc[tournamentCounts["counts"] > threshold]
  
  
  
  result = data.merge(bigTournaments, how = "inner", on = "TournamentID")
  result.drop(["counts"],axis=1)
  
  dropTournaments = tournamentCounts.loc[tournamentCounts["counts"] < threshold]
  dropped = dropTournaments["TournamentID"].tolist()
  
  print("Dropped Tournaments")
  for x in dropped:
    print(x)
    
  return(result)

In [0]:
RoundsDf = dropSmallTournaments(RoundsDf,100)

Dropped Tournaments
373
412
426
448
456
457
461

In [0]:
RoundsDf.groupby("TournamentID").size().reset_index(name="counts")

,TournamentID,counts
0,375,218
1,385,272
2,403,204
3,406,120
4,408,146
5,409,248
6,410,241
7,413,188
8,414,198
9,415,164


In [0]:
#RoundsDf.to_csv('/dbfs/FileStore/karbide/Rounds.txt')

In [0]:
testRoundDf = pd.read_csv('/dbfs/FileStore/karbide/Rounds.txt')

In [0]:
pStats = pd.read_csv('/dbfs/FileStore/karbide/pga_tour_stats_2020.csv')

In [0]:
pStats.columns

Out[8]: Index(['PLAYER NAME', 'AVG_APPROACH_BIRDIE_OR_BETTER',
 'AVG_APPROACH_BOGEY_OR_WORSE', 'AVG_APPROACH_PAR',
 'AVG_APPROACH_SHOT_DISTANCE', 'FAIRWAY_APPROACH_RTP',
 'GIR_PCT_FAIRWAY_BUNKER', 'GIR_PCT_FAIRWAY', 'GIR_PCT_OTHER',
 'GIR_PCT_OVERALL', 'GIR_PCT_100-125', 'GIR_PCT_OVER_100',
 'GIR_PCT_125-150', 'GIR_PCT_150-175', 'GIR_PCT_175-200',
 'GIR_PCT_OVER_200', 'GIR_PCT_75-100', 'GIR_PCT_UNDER_100',
 'GIR_PCT_UNDER_125', 'GIR_PCT_UNDER_75', 'GREEN_PCT_SAND_SAVES',
 'GREEN_PCT_SCRAMBLE', 'GREEN_PCT_SCRAMBLE_FRINGE',
 'GREEN_PCT_SCRAMBLE_ROUGH', 'GREEN_PCT_SCRAMBLE_SAND',
 'FINISHES_FEDEXCUP_BONUS', 'FINISHES_OFFICIAL_MONEY', 'FINISHES_TOP10',
 'FINISHES_OFFICIAL_UNOFFICIAL_MONEY', 'TEE_AVG_APEX',
 'TEE_AVG_BALL_SPEED', 'TEE_BALL_STRIKING', 'TEE_AVG_CARRY',
 'TEE_LONGEST_CARRY_DISTANCE', 'TEE_AVG_CARRY_EFF',
 'TEE_AVG_CLUB_HEAD_SPEED', 'TEE_AVG_DISTANCE_FROM_CENTER_FR',
 'TEE_DRIVING_ACCURACY_PCT', 'TEE_AVG_DRIVING_DISTANCE',
 'TEE_AVG_FAIRWAY_BUNKER_TENDENCY_PCT', 'TEE_AVG_FIR_PCT',
 'TEE_AVG_LAUNCH_ANGLE', 'TEE_AVG_LEFT_ROUGH_TENDENCY_PCT',
 'TEE_LONGEST_DRIVES', 'TEE_AVG_RIGHT_ROUGH_TENDENCY_PCT',
 'TEE_AVG_SMASH_FACTOR', 'TEE_AVG_SPIN_RATE', 'PUTTING_AVG_ONE_PUTTS',
 'PUTTING_AVG_TWO_PUTTS', 'PUTTING_AVG_THREE_PUTTS',
 'PUTTING_APPROACH_PERFORMANCE', 'PUTTING_AVG_DIST_BIRDIE',
 'PUTTING_AVG_DIST_EAGLE', 'PUTTING_AVG_DIST_MADE',
 'PUTTING_PCT_BIRDIE_OR_BETTER', 'PUTTING_AVG_OVERALL',
 'PUTTING_AVG_PUTTS', 'SCORING_AVG_BIRDIES',
 'SCORING_PCT_BIRDIE_OR_BETTER', 'SCORING_BIRDIE_BOGEY_RATIO',
 'SCORING_BOGEY_AVERAGE', 'SCORING_EAGLES_PER_HOLE',
 'SCORING_PAR3_AVERAGE', 'SCORING_PAR4_AVERAGE', 'SCORING_PAR5_AVERAGE',
 'SCORING_ROUNDS_IN_60S', 'SCORING_AVG_OVERALL', 'SCORING_TOTAL_BIRDIES',
 'SCORING_TOTAL_EAGLES', 'STREAKS_CUTS_MADE', 'STREAKS_FIRS_MADE',
 'STREAKS_GIRS_MADE', 'STREAKS_HOLES_UNDER_PAR'],
 dtype='object')

In [0]:
pStats.describe()

,AVG_APPROACH_BIRDIE_OR_BETTER,AVG_APPROACH_BOGEY_OR_WORSE,AVG_APPROACH_PAR,AVG_APPROACH_SHOT_DISTANCE,FAIRWAY_APPROACH_RTP,GIR_PCT_FAIRWAY_BUNKER,GIR_PCT_FAIRWAY,GIR_PCT_OTHER,GIR_PCT_OVERALL,GIR_PCT_100-125,GIR_PCT_OVER_100,GIR_PCT_125-150,GIR_PCT_150-175,GIR_PCT_175-200,GIR_PCT_OVER_200,GIR_PCT_75-100,GIR_PCT_UNDER_100,GIR_PCT_UNDER_125,GIR_PCT_UNDER_75,GREEN_PCT_SAND_SAVES,GREEN_PCT_SCRAMBLE,GREEN_PCT_SCRAMBLE_FRINGE,GREEN_PCT_SCRAMBLE_ROUGH,GREEN_PCT_SCRAMBLE_SAND,FINISHES_FEDEXCUP_BONUS,FINISHES_TOP10,TEE_AVG_BALL_SPEED,TEE_BALL_STRIKING,TEE_AVG_CARRY,TEE_LONGEST_CARRY_DISTANCE,TEE_AVG_CARRY_EFF,TEE_AVG_CLUB_HEAD_SPEED,TEE_DRIVING_ACCURACY_PCT,TEE_AVG_DRIVING_DISTANCE,TEE_AVG_FAIRWAY_BUNKER_TENDENCY_PCT,TEE_AVG_FIR_PCT,TEE_AVG_LAUNCH_ANGLE,TEE_AVG_LEFT_ROUGH_TENDENCY_PCT,TEE_LONGEST_DRIVES,TEE_AVG_RIGHT_ROUGH_TENDENCY_PCT,TEE_AVG_SMASH_FACTOR,TEE_AVG_SPIN_RATE,PUTTING_AVG_ONE_PUTTS,PUTTING_AVG_TWO_PUTTS,PUTTING_AVG_THREE_PUTTS,PUTTING_PCT_BIRDIE_OR_BETTER,PUTTING_AVG_OVERALL,PUTTING_AVG_PUTTS,SCORING_AVG_BIRDIES,SCORING_PCT_BIRDIE_OR_BETTER,SCORING_BIRDIE_BOGEY_RATIO,SCORING_BOGEY_AVERAGE,SCORING_EAGLES_PER_HOLE,SCORING_PAR3_AVERAGE,SCORING_PAR4_AVERAGE,SCORING_PAR5_AVERAGE,SCORING_ROUNDS_IN_60S,SCORING_AVG_OVERALL,SCORING_TOTAL_BIRDIES,SCORING_TOTAL_EAGLES,STREAKS_CUTS_MADE,STREAKS_FIRS_MADE,STREAKS_GIRS_MADE,STREAKS_HOLES_UNDER_PAR
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,1.650000e+02,177.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,82.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,135.000000,356.000000,356.000000,356.000000
mean,175.308173,163.554327,161.907212,165.028846,-0.098341,48.938942,78.128558,53.015577,67.054279,75.863029,62.814038,69.884952,64.916442,56.213654,46.186106,80.620962,85.472452,81.759856,87.431154,49.608077,59.126442,89.916923,57.217885,50.690048,3.728182e+05,2.423729,170.314615,195.403846,280.158654,340.787500,2.458514,113.977404,60.566683,296.368750,6.150481,60.547644,10.471010,14.515048,370.677885,15.170481,1.495062,2533.982212,7.074663,10.189038,0.021707,30.632452,1.610587,28.990962,3.781731,21.517404,1.388750,2.505048,5.370192,3.050337,4.015288,4.638798,25.658654,70.874928,224.802885,5.370192,3.814815,1.553371,1.601124,3.561798
std,6.215221,5.282318,3.182591,3.200956,0.035058,9.174638,2.762681,3.539991,2.569055,5.195936,3.231113,4.575131,5.374336,4.953888,5.798943,7.218627,3.176150,3.040734,3.294634,6.761254,3.489676,5.281595,5.326012,7.467329,1.291266e+06,1.949971,5.946292,93.662207,10.209677,110.182748,0.044383,4.206966,5.019827,8.448773,1.217386,5.064885,1.488102,2.861394,24.131025,2.915349,0.011697,186.105993,0.413774,0.376903,0.009914,2.197730,0.027395,0.493105,0.274874,1.585288,0.215327,0.296532,3.009241,0.049513,0.040586,0.067265,8.981668,0.765909,57.045067,3.009241,2.589258,2.217717,2.086415,1.207536
min,152.900000,146.600000,148.700000,154.800000,-0.205000,29.000000,64.310000,43.240000,52.050000,52.810000,44.350000,55.970000,33.860000,40.400000,28.400000,59.180000,73.810000,70.000000,78.460000,25.860000,48.760000,73.910000,44.440000,26.090000,7.000000e+04,1.000000,157.980000,14.000000,253.800000,285.900000,2.313000,105.240000,46.810000,277.900000,3.000000,45.780000,5.960000,7.490000,304.000000,8.860000,1.445000,2070.900000,5.740000,9.230000,0.010000,24.810000,1.549000,27.880000,2.890000,16.790000,0.620000,1.790000,1.000000,2.930000,3.890000,4.450000,2.000000,68.978000,107.000000,1.000000,2.000000,0.000000,0.000000,1.000000
25%,171.475000,160.100000,159.900000,163.200000,-0.11925

In [0]:
# I'm going to select ~30 of these stats, then later check for independence 
pStatsKeep = pStats[['PLAYER NAME',"GIR_PCT_FAIRWAY_BUNKER",	"GIR_PCT_FAIRWAY", "GIR_PCT_OVERALL", 'GIR_PCT_OVER_100', 'GIR_PCT_OVER_200', 'GIR_PCT_UNDER_100', 'GREEN_PCT_SCRAMBLE_SAND', 'GREEN_PCT_SCRAMBLE_ROUGH', 'FINISHES_TOP10', 'TEE_AVG_BALL_SPEED', 'TEE_AVG_DRIVING_DISTANCE', 'TEE_DRIVING_ACCURACY_PCT','TEE_AVG_LAUNCH_ANGLE', 'TEE_AVG_LEFT_ROUGH_TENDENCY_PCT', 'TEE_AVG_RIGHT_ROUGH_TENDENCY_PCT', 'TEE_AVG_SPIN_RATE', 'PUTTING_AVG_ONE_PUTTS',
       'PUTTING_AVG_TWO_PUTTS', 'PUTTING_AVG_DIST_BIRDIE', "PUTTING_AVG_PUTTS"]]

In [0]:
# Average Birdie Putt Distance is currently in feet and inches, can we change this to just inches
def split_Dist(item):
  if item != "nan":
    spDist = item.split("' ")
    ft_ = float(spDist[0])
    in_ = float(spDist[1].replace("\"",""))
    return (12*ft_) + in_


In [0]:
pStatsKeep = pStatsKeep.astype({"PUTTING_AVG_DIST_BIRDIE":"str"})


pStatsKeep["PUTTING_AVG_DIST_BIRDIE_INCH"] = pStatsKeep["PUTTING_AVG_DIST_BIRDIE"].apply(lambda x:split_Dist(x))

In [0]:
pStatsKeep.describe()

,GIR_PCT_FAIRWAY_BUNKER,GIR_PCT_FAIRWAY,GIR_PCT_OVERALL,GIR_PCT_OVER_100,GIR_PCT_OVER_200,GIR_PCT_UNDER_100,GREEN_PCT_SCRAMBLE_SAND,GREEN_PCT_SCRAMBLE_ROUGH,FINISHES_TOP10,TEE_AVG_BALL_SPEED,TEE_AVG_DRIVING_DISTANCE,TEE_DRIVING_ACCURACY_PCT,TEE_AVG_LAUNCH_ANGLE,TEE_AVG_LEFT_ROUGH_TENDENCY_PCT,TEE_AVG_RIGHT_ROUGH_TENDENCY_PCT,TEE_AVG_SPIN_RATE,PUTTING_AVG_ONE_PUTTS,PUTTING_AVG_TWO_PUTTS,STREAKS_CUTS_MADE,STREAKS_FIRS_MADE,STREAKS_GIRS_MADE,STREAKS_HOLES_UNDER_PAR,PUTTING_AVG_DIST_BIRDIE_INCH
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,177.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,135.000000,356.000000,356.000000,356.000000,208.000000
mean,48.938942,78.128558,67.054279,62.814038,46.186106,85.472452,50.690048,57.217885,2.423729,170.314615,296.368750,60.566683,10.471010,14.515048,15.170481,2533.982212,7.074663,10.189038,3.814815,1.553371,1.601124,3.561798,116.072115
std,9.174638,2.762681,2.569055,3.231113,5.798943,3.176150,7.467329,5.326012,1.949971,5.946292,8.448773,5.019827,1.488102,2.861394,2.915349,186.105993,0.413774,0.376903,2.589258,2.217717,2.086415,1.207536,9.672891
min,29.000000,64.310000,52.050000,44.350000,28.400000,73.810000,26.090000,44.440000,1.000000,157.980000,277.900000,46.810000,5.960000,7.490000,8.860000,2070.900000,5.740000,9.230000,2.000000,0.000000,0.000000,1.000000,95.000000
25%,40.725000,76.630000,65.295000,60.672500,42.592500,83.170000,45.987500,53.932500,1.000000,165.907500,290.050000,57.365000,9.317500,12.505000,13.060000,2424.400000,6.807500,9.927500,2.000000,0.000000,0.000000,3.000000,109.000000
50%,49.150000,78.360000,67.195000,63.205000,46.025000,85.825000,50.895000,57.295000,2.000000,169.850000,296.450000,60.750000,10.755000,14.500000,15.040000,2490.950000,7.130000,10.230000,3.000000,1.000000,1.000000,4.000000,116.000000
75%,55.450000,79.807500,68.545000,64.710000,49.417500,87.825000,55.285000,61.125000,3.000000,174.032500,301.725000,63.360000,11.762500,16.500000,17.137500,2655.450000,7.355000,10.400000,5.000000,2.000000,2.000000,4.000000,121.000000
max,72.700000,85.330000,74.220000,71.270000,64.410000,92.310000,67.690000,74.650000,10.000000,189.180000,322.100000,74.500000,13.910000,22.680000,24.020000,3088.700000,8.170000,11.540000,19.000000,15.000000,11.000000,9.000000,146.000000


In [0]:
pStatsKeep["FINISHES_TOP10"].fillna(0,inplace = True)
pStatsKeep.dropna(inplace = True)
pStatsKeep.drop_duplicates(inplace = True)

In [0]:
pStatsKeep.groupby("PLAYER NAME").agg({"GIR_PCT_FAIRWAY_BUNKER": "count"}).sort_values("GIR_PCT_FAIRWAY_BUNKER", ascending = False).head(3)
# we still have 8 zach johnsons so lets dump him

,GIR_PCT_FAIRWAY_BUNKER
PLAYER NAME,
Zach Johnson,8
Aaron Baddeley,1
Patrick Rodgers,1


In [0]:
pStatsKeep = pStatsKeep.loc[pStatsKeep["PLAYER NAME"] != "Zach Johnson"]

In [0]:
#Now we have to add Player IDs

PlayerNames = pd.read_csv("/dbfs/FileStore/karbide/Players.txt")
PlayerNames = PlayerNames[["DraftKingsName","PlayerID"]]

In [0]:
pStatsKeepIDs = pStatsKeep.merge(PlayerNames, how = "left", left_on = "PLAYER NAME", right_on = "DraftKingsName")

In [0]:
pStatsKeepIDsDropped = pStatsKeepIDs.loc[pStatsKeepIDs["PlayerID"].isna()]

In [0]:
pStatsKeepIDsDropped["PLAYER NAME"]

Out[166]: 5 Ted Potter, Jr.
25 Fabián Gómez
36 William Gordon
74 Kristoffer Ventura
136 Matthew Fitzpatrick
140 Cheng Tsung Pan
161 Sebastián Muñoz
Name: PLAYER NAME, dtype: object

In [0]:
#again, we have to manually adjust these names

In [0]:
PlayerNames2 = pd.read_csv("/dbfs/FileStore/karbide/Players.txt")
PlayerNames2 = PlayerNames2[["DraftKingsName","PlayerID","FirstName","LastName"]]

In [0]:
print(PlayerNames2.loc[PlayerNames2["FirstName"] == "Ted"])
pStatsKeepIDs.at[5,"PlayerID"] = 40001173


DraftKingsName PlayerID FirstName LastName
548 NaN 40000183 Ted Brown
3529 Ted Potter Jr. 40001173 Ted Potter Jr
3569 Ted Purdy 40001187 Ted Purdy
3898 NaN 40001287 Ted Schulz
4059 Ted Smith 40002287 Ted Smith
4386 Ted Tryba 40001464 Ted Tryba

In [0]:
print(PlayerNames2.loc[PlayerNames2["FirstName"] == "Fabian"])
pStatsKeepIDs.at[25,"PlayerID"] = 40000514

DraftKingsName PlayerID FirstName LastName
345 Fabian Becker 40003223 Fabian Becker
1519 Fabian Gomez 40000514 Fabian Gomez

In [0]:
print(PlayerNames2.loc[PlayerNames2["LastName"] == "Gordon"])
pStatsKeepIDs.at[36,"PlayerID"] = 40003663

DraftKingsName PlayerID FirstName LastName
1534 Will Gordon 40003663 Will Gordon

In [0]:
print(PlayerNames2.loc[PlayerNames2["LastName"] == "Ventura"])
pStatsKeepIDs.at[74,"PlayerID"] = 40003179

DraftKingsName PlayerID FirstName LastName
4478 Kris Ventura 40003179 Kristoffer Ventura

In [0]:
print(PlayerNames2.loc[PlayerNames2["LastName"] == "Fitzpatrick"])
pStatsKeepIDs.at[136,"PlayerID"] = 40000430

DraftKingsName PlayerID FirstName LastName
1284 Alex Fitzpatrick 40003219 Alex Fitzpatrick
1285 Matt Fitzpatrick 40000430 Matthew Fitzpatrick

In [0]:
print(PlayerNames2.loc[PlayerNames2["LastName"] == "Pan"])
pStatsKeepIDs.at[140,"PlayerID"] = 40001109

DraftKingsName PlayerID FirstName LastName
3337 C.T. Pan 40001109 Cheng Tsung Pan

In [0]:
print(PlayerNames2.loc[PlayerNames2["FirstName"] == "Sebastian"])
pStatsKeepIDs.at[161,"PlayerID"] = 40001682

DraftKingsName PlayerID FirstName LastName
651 Sebastian Cappelen 40000218 Sebastian Cappelen
892 Sebastian Crampton 40003831 Sebastian Crampton
1354 NaN 40004968 Sebastian Friedrichsen
1701 NaN 40003241 Sebastian Hansson
1764 Sebastian Heisele 40002630 Sebastian Heisele
2702 NaN 40001774 Sebastian Maclean
3077 Sebastian Munoz 40001682 Sebastian Munoz
3459 NaN 40004963 Sebastian Petersen
3494 NaN 40001712 Sebastian Pinzon
3787 Sebastian Saavedra 40001253 Sebastian Saavedra
3807 NaN 40002009 Sebastian Salem
4074 Sebastian Soderberg 40002360 Sebastian Söderberg
4246 NaN 40002292 Sebastian Szirmak
4468 Sebastian Vazquez 40001493 Sebastian Vazquez
4487 Sebastian Vida 40004921 Sebastian Vida
4696 Sebastian Wittmann 40004184 Sebastian Wittmann

In [0]:
sum(pStatsKeepIDs["PlayerID"].isna())
#now every player has a name and ID
#I might consider runnning the API pull again for this list of players

Out[175]: 0

In [0]:
pStatsKeepIDs.drop(["DraftKingsName"], axis = 1, inplace = True)

In [0]:
pStatsKeepIDs = pStatsKeepIDs.astype({"PlayerID":"int"})

In [0]:
#pStatsKeepIDs.to_csv('/dbfs/FileStore/karbide/PlayerStats.txt')

In [0]:
StatPlayers = pStatsKeepIDs["PlayerID"].tolist()

In [0]:
#RoundsDf, HolesDf = allPlayers(StatPlayers,Tourney_IDs,"c76c6101adbf4b0abb54a7a6eb5ddbb4")

/local_disk0/tmp/1638023672513-0/PythonShell.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

 from six.moves import range
/local_disk0/tmp/1638023672513-0/PythonShell.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

 from py4j.java_collections import ListConverter
/local_disk0/tmp/1638023672513-0/PythonShell.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

 import resource

In [0]:
RoundsDf = dropSmallTournaments(RoundsDf,100)

Dropped Tournaments

In [0]:
#RoundsDf.to_csv('/dbfs/FileStore/karbide/Rounds.txt')
#HolesDf.to_csv('/dbfs/FileStore/karbide/Holes.txt')

In [0]:
StrokesGained = pd.read_csv("/dbfs/FileStore/karbide/StrokesGained.csv", encoding = 'latin-1')

In [0]:
StrokesGained.head()
print(StrokesGained.shape())

,PLAYER NAME,AVERAGE,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P
0,Jon Rahm,2.098,134.286,113.113,21.174
1,Bryson DeChambeau,1.823,123.959,92.967,30.993
2,Patrick Cantlay,1.820,123.758,96.411,27.349
3,Louis Oosthuizen,1.599,95.967,50.115,45.852
4,Xander Schauffele,1.484,93.464,63.250,30.215


In [0]:
pIDs = pStatsKeepIDs[["PLAYER NAME", "PlayerID"]]
StrokesGainedIDs = StrokesGained.merge(pIDs, how = "inner", on = "PLAYER NAME")

In [0]:
#StrokesGainedIDs.to_csv("/dbfs/FileStore/karbide/StrokesGainedIDs.txt")